# python中的socket网络编程
非原创

2017.03.26
与C++或Java相比，利用python来进行网络编程是十分方便的，下面我们展示两种socket的简单demo：传送字符串和传送文件
### socket传送字符串
(1) 运行socket\_server.py文件, 内容如下：
```python
# encoding: utf8
import sys
import socket

def main():	
	skt = socket.socket()
	host = socket.gethostname()
	port = 12345
	skt.bind((host, port))
	skt.listen(5)
	while True:
		c, addr = skt.accept()
		print 'connected address:', addr
		c.send('welcome to us')
		c.close()

if __name__ == '__main__':
	main()
```

(2) 运行socket\_client.py文件， 内容如下:
```python
# encoding: utf-8
import sys
import socket

def main():
	skt = socket.socket()
	host = socket.gethostname()
	port = 12345
	skt.connect((host, port))
	print skt.recv(1024)
	skt.close()

if __name__ == '__main__':
	main()
```

(3) 通过观察两个程序的控制台，可以发现字符串传送成功

### socket传送文件
(1) 先运行recv.py文件， 文件内容如下：
```python
# encoding: utf8

import socket
import threading
import time
import sys
import os
import struct


def socket_service():
	try:
		s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
		s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
		s.bind(('127.0.0.1', 6666))
		s.listen(10)
	except socket.error as msg:
		print msg
		sys.exit(1)
	print 'Waiting connection...'
	
	while 1:
		conn, addr = s.accept()
		t = threading.Thread(target=deal_data, args=(conn, addr))
		t.start()


def deal_data(conn, addr):
	print 'Accept new connection from {0}'.format(addr)
	#conn.settimeout(500)
	conn.send('Hi, Welcome to the server!')
	
	while 1:
		fileinfo_size = struct.calcsize('128sl')
		buf = conn.recv(fileinfo_size)
		if buf:
			filename, filesize = struct.unpack('128sl', buf)
			fn = filename.strip('\00')
			new_filename = os.path.join('./', 'new_' + fn)
			print 'file new name is {0}, filesize if {1}'.\
			format(new_filename,
			filesize)
			
			recvd_size = 0  # 定义已接收文件的大小
			fp = open(new_filename, 'wb')
			print 'start receiving...'
			
			while not recvd_size == filesize:
				if filesize - recvd_size > 1024:
					data = conn.recv(1024)
					recvd_size += len(data)
				else:
					data = conn.recv(filesize - recvd_size)
					recvd_size = filesize
				fp.write(data)
			fp.close()
			print 'end receive...'
		conn.close()
		break


if __name__ == '__main__':
socket_service()
```
(2) 运行send.py文件，文件内容如下:
```python
# encoding: utf8

import socket
import os
import sys
import struct

def socket_client():
	try:
		s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
		s.connect(('127.0.0.1', 6666))
	except socket.error as msg:
		print msg
		sys.exit(1)
	print s.recv(1024)
	
	while 1:
	filepath = raw_input('please input file path: ')
	if os.path.isfile(filepath):
		# 定义定义文件信息。128s表示文件名为128bytes长，
		# l表示一个int或log文件类型，在此为文件大小
		fileinfo_size = struct.calcsize('128sl')
		# 定义文件头信息，包含文件名和文件大小
		fhead = struct.pack('128sl', os.path.basename(filepath),
		os.stat(filepath).st_size)
		s.send(fhead)
		print 'client filepath: {0}'.format(filepath)
		
		fp = open(filepath, 'rb')
		while 1:
			data = fp.read(1024)
			if not data:
				print '{0} file send over...'.format(filepath)
				break
			s.send(data)
	s.close()
	break

if __name__ == '__main__':
	socket_client()
```
(3) 在send.py的控制台钟输入文件路径，然后在recv.py的所在目录下就会发现以“new_”为前缀的接收文件